In [1]:
import sys
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['OPENAI_API_KEY'] = 'zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

sys.path.append('/local/nlp/milad/code/explainability-of-art-images/')


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

import utils

/local/nlp/milad/conda-envs/art-images/lib/python3.9/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
model_name = "Qwen/Qwen2.5-VL-7B-Instruct"  # or use "Qwen/Qwen2.5-VL-32B-Instruct"

# Use `trust_remote_code=True` for both processor and model
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Qwen2_5_VLForConditionalGeneration(
  (model): Qwen2_5_VLModel(
    (visual): Qwen2_5_VisionTransformerPretrainedModel(
      (patch_embed): Qwen2_5_VisionPatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2_5_VLVisionBlock(
          (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
          (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
          (attn): Qwen2_5_VLVisionSdpaAttention(
            (qkv): Linear(in_features=1280, out_features=3840, bias=True)
            (proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): Qwen2_5_VLMLP(
            (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
            (act_fn): Si

### Styles of Arches in Architecture

In [16]:
def answer_questions(image_question_pairs):
    # Generate answers
    for image_path, question in image_question_pairs:
        image = Image.open(image_path).convert("RGB")
    
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image,
                    },
                    {"type": "text", "text": question},
                ],
            }
        ]
        
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")
    
        with torch.no_grad():
            generated_ids = model.generate(**inputs, do_sample=True, max_new_tokens=200)

        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )[0]
        print(f"Q: {question}\nA: {output_text}\n")
        print('==============')

In [17]:
image_question_pairs = [
    ('./data/sample_images/arches_1.png', 'What is this image?'),
    ('./data/sample_images/arches_3.png', 'What is the difference between these two arches?')
]

In [18]:
answer_questions(image_question_pairs)

Q: What is this image?
A: This image shows a section of an architectural facade featuring a series of arched openings supported by columns. The arches are pointed and have decorative elements at the top, which suggest a style that could be influenced by Islamic or Moorish architecture. The columns appear to be fluted, a common feature in classical and Renaissance architecture, but the overall design has a distinct Middle Eastern aesthetic. This type of architecture is often found in historical buildings in regions such as Spain, North Africa, or the Middle East. The specific style and origin would depend on more context, such as the location of the building or additional architectural details.

Q: What is the difference between these two arches?
A: The two arches in the image appear to be similar in basic structure, both featuring a pointed top and a rounded base. However, there are some subtle differences:

1. **Design Details**: The arch on the left (labeled as 2) has a more ornate d